In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from scipy import *
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy
from scipy import spatial
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
STOP_WORDS = stopwords.words("english")
from sklearn.model_selection import train_test_split
import xgboost as xgb
import sklearn.metrics as classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve, auc, roc_curve ,f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/Capstone Data/train.csv") # importing the dataset

df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))
df.head()

In [ ]:
df.head(20)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
documentA = 'the man went out for a walk'
documentB = 'the children sat around the fire'

In [ ]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df7 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
print(df7.head())

In [ ]:
print(vectors)

In [ ]:
print(feature_names)

In [ ]:
df['question1']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
question = list(df['question1']) + list(df['question2'])

questions = [ques for ques in question if ques not in STOP_WORDS]

tfidf = TfidfVectorizer(lowercase=False, )
x=tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [ ]:
print(x) # first id , feautre_names no. , tfidf score

In [ ]:
tfidf.idf_